In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Загрузка предобученной модели и токенизатора
model_name = "cross-encoder/nli-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Переводим модель в режим оценки, т.к. больше не будем её обучать, и нам необходимы только её предсказания.
model.eval()

text = "Как настроить CI/CD для проекта на GitLab?"
candidate_labels = ["DevOps", "IT", "Frontend", "Backend", "Data Science", "Machine Learning", "Cybersecurity", "Cloud Computing", "Mobile Development", "Game Development", "Database Administration"]

#Создаём пример входных данных.
# Для данной модели токенизатор возвращает тензор, содержащий идентификаторы токенов для текста, и тензор, указывающий, какие токены следует учитывать во входных данных.
inputs = tokenizer([text] * 11, candidate_labels, padding=True, truncation=True, return_tensors="pt")

fileForSavingTheModel = 'model.onnx'
torch.onnx.export(
    model = model, # Какую модель сериализуем.
    args = (inputs['input_ids'], inputs['attention_mask']), # Пример аргументов, принимаемых моделью, чтобы она +- понимала, какие тензоры ей ожидать.
    f = fileForSavingTheModel, # Путь к сохранению сериализованной модели.
    input_names = ["input_ids", "attention_mask"], # Модель принимает два тензора.
    output_names = ["logits"], # Модель возвращает сырые логиты, по 3 на каждую категорию:
    #[следуют друг из друга, противоречат друг другу, никак не связаны]
    dynamic_axes={  # Указание динамических осей.
        "input_ids": {0: "batch_size", 1: "sequence_length"},  # Т.к. 0 - может поступить сразу несколько категорий, и их данные могут отличаться, 1 - данные могут быть разной длины.
        "attention_mask": {0: "batch_size", 1: "sequence_length"},  # Аналогично с input_ids.
        "logits": {0: "batch_size"}  # Т.к. может поступить несколько категорий, значит понадобится вывести несколько ответов.
    },
)
print("Модель успешно сериализована в", fileForSavingTheModel)
tokenizer.save_pretrained("tokenizer")
print("Токенайзер успешно сохранён")


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Модель успешно сериализована в model.onnx
Токенайзер успешно сохранён
